In [ ]:
%pip install face_recognition efficientnet tensorflow opencv-python-headless gradio kaggle


In [ ]:
import os
import cv2
import numpy as np
import random
import gradio as gr
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from pathlib import Path
from PIL import Image
from tensorflow.keras.models import load_model


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d manjilkarki/deepfake-and-real-images
!unzip -q deepfake-and-real-images.zip -d dataset


Dataset URL: https://www.kaggle.com/datasets/manjilkarki/deepfake-and-real-images
License(s): unknown
 99% 1.67G/1.68G [00:23<00:00, 319MB/s]
100% 1.68G/1.68G [00:23<00:00, 77.8MB/s]


In [ ]:
def load_images(folder, label, size=(244, 244)):
    data = []
    for filename in os.listdir(folder):
        path = os.path.join(folder, filename)
        img = cv2.imread(path)
        if img is not None:
            img = cv2.resize(img, size)
            data.append((img, label))
    return data

In [ ]:
print("GPU available:", tf.config.list_physical_devices('GPU'))

# dataset link --   https://www.kaggle.com/datasets/manjilkarki/deepfake-and-real-images

GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
train_real = "/content/dataset/Dataset/Train/Real"
train_fake = "/content/dataset/Dataset/Train/Fake"

test_real = "/content/dataset/Dataset/Test/Real"
test_fake = "/content/dataset/Dataset/Test/Fake"

val_real = "/content/dataset/Dataset/Validation/Real"
val_fake = "/content/dataset/Dataset/Validation/Fake"


In [ ]:
train_data = load_images(train_real, 0) + load_images(train_fake, 1)
test_data = load_images(test_real, 0) + load_images(test_fake, 1)
val_data = load_images(val_real, 0) + load_images(val_fake, 1)


In [ ]:
# Load data
real_path = "/content/dataset/Dataset/Train/Real"
fake_path = "/content/dataset/Dataset/Train/Fake"

real_data = load_images(real_path, 0)
fake_data = load_images(fake_path, 1)
data = real_data + fake_data

In [ ]:


# Shuffle
random.shuffle(data)

# Convert to NumPy with reduced memory usage
X = np.array([item[0] for item in data], dtype=np.float32) / 255.0
y = np.array([item[1] for item in data])
y = to_categorical(y, num_classes=2)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)


In [ ]:
base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(244, 244, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(
    datagen.flow(X_train, y_train, batch_size=16),
    epochs=5,
    validation_data=(X_test, y_test)
)
model.save("deepfake_detector_efficientnetb4.h5")

In [ ]:
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Step 4: Recompile with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Step 5: Continue training (fine-tuning)
model.fit(
    datagen.flow(X_train, y_train, batch_size=16),
    epochs=5,
    validation_data=(X_test, y_test)
)

# Step 6: Save the fine-tuned model
model.save("deepfake_detector_efficientnetb4_finetuned.h5")

In [ ]:

known_faces = []
known_names = []

dataset_path = Path(r"D:\new_project\face_dataset")

# Accept common image file extensions
valid_extensions = {'.jpg', '.jpeg', '.png', '.bmp'}

for img_path in dataset_path.rglob("*"):
    if img_path.suffix.lower() not in valid_extensions:
        continue

    try:
        image = face_recognition.load_image_file(str(img_path))
        encodings = face_recognition.face_encodings(image)

        if encodings:
            known_faces.append(encodings[0])
            name = img_path.parent.name  # Use folder name as label
            known_names.append(name)
        else:
            print(f"No face found in: {img_path.name}")

    except Exception as e:
        print(f"Error processing {img_path.name}: {e}")


In [ ]:

# Load your trained model before prediction
model = load_model("deepfake_detector_efficientnetb4_finetuned.h5")

def predict_and_recognize(image, is_webcam=False):
    try:
        # Deepfake Detection (always performed)
        # Webcam input is resized and processed for deepfake detection
        image_resized = cv2.resize(image, (244, 244)) / 255.0
        input_tensor = np.expand_dims(image_resized, axis=0)

        pred = model.predict(input_tensor)[0]
        predicted_class = np.argmax(pred)
        confidence = round(float(np.max(pred)) * 100, 2)

        is_real = predicted_class == 0 and confidence >= 50.0  # Adjusted threshold slightly
        result = "Real" if is_real else "Fake"
        full_result = f"{result} Face ({confidence}% Confidence)"

        # Removed Face recognition part due to CUDA error with face_recognition library

        return full_result # Return only deepfake detection result

    except Exception as e:
        return f"Error during prediction: {str(e)}"

In [ ]:
# Load your trained model before prediction (re-added here in case it was removed elsewhere)
# model = load_model("deepfake_detector_efficientnetb4_finetuned.h5") # Assuming model is loaded in a previous cell

interface = gr.Interface(fn=predict_and_recognize,
                         inputs=gr.Image(type="numpy", label="Upload or Capture Face Image"),
                         outputs="text",
                         title="Deepfake Detection", # Updated title
                         description="Upload a face image or use your webcam to detect if it's a deepfake.") # Updated description

interface.launch(share=True)